In [1]:
import pandas as pd

In [2]:

customers_df = pd.read_csv('../data/hm/customers.csv')
article_df = pd.read_csv('../data/hm/articles.csv', dtype={'article_id': str})
transactions_df = pd.read_csv('../data/hm/transactions_train.csv', dtype={'article_id': str})

In [3]:
article_df = article_df[['article_id', 'prod_name', 'product_type_name', 'colour_group_name', 'section_name']]
customers_df = customers_df[['customer_id', 'age']]

In [4]:
df = pd.merge(transactions_df, article_df, how='inner', on='article_id')

customer_count = df.customer_id.value_counts()
valid_customers = customer_count[5000:15000]
valid_customer_ids = set(valid_customers.index)

valid_customer_mask = df.customer_id.isin(valid_customer_ids)

a = df.article_id.value_counts()
too_popular_articles = set(a[:50].index)
too_unpopular_articles = set(a[a < 20].index)

exclude_article_mask = df.article_id.isin(too_popular_articles | too_unpopular_articles)

mask = (valid_customer_mask & (~exclude_article_mask))

filtered_df = df[mask]

In [5]:
from tqdm import tqdm
tqdm.pandas()

raw_sequences = []
customer_ids = []
seq_labels = None

for customer_id, personal_df in tqdm(filtered_df.groupby('customer_id')):
    raw_sequences.append(list(personal_df.article_id.values))
    customer_ids.append(customer_id)

100%|██████████| 10000/10000 [00:00<00:00, 16218.08it/s]

2044684


In [ ]:
filtered_articles = filtered_df.article_id.unique()

In [ ]:
filtered_articles_df = article_df[article_df.article_id.isin(set(filtered_articles))]
filtered_articles_df = filtered_articles_df.set_index('article_id')
filtered_articles_df.to_csv('../data/hm/items.csv')

In [ ]:
item_list = []

for index, row in filtered_articles_df.iterrows():
    item_name = row.prod_name + ' - ' + row.product_type_name
    item_list.append(item_name)

In [ ]:
raw_sequences = [' '.join(sequence) for sequence in raw_sequences]

In [ ]:
sequence_df = pd.DataFrame(raw_sequences, columns=['sequence'])
sequence_df['customer_id'] = customer_ids
sequence_df.to_csv('../data/hm/filtered_purchase_history.csv', index=False)

In [65]:
items = pd.read_csv("../data/hm/items.csv", dtype={"article_id": str}, index_col='article_id')

In [56]:
customers_df.index = customers_df.customer_id
customers_df = customers_df.drop("customer_id", axis=1)

In [62]:
customers_df.to_csv('../data/hm/customers.csv')

In [3]:
pd.read_csv('../data/hm/filtered_purchase_history.csv')

,sequence,customer_id
0,0682550001 0677999001 0677999001 0688873005 06...,00080403a669b3b89d1bef1ec73ea466d95e39698d6dde...
1,0464277014 0608458001 0608458001 0496111020 06...,00201f34c8c92683263346d78c2b45ffd0c6927229542d...
2,0553139001 0632143001 0629764001 0355569001 03...,0036a44bd648ce2dbc32688a465b9628b7a78395302f26...
3,0567475001 0448515001 0448515001 0637255001 06...,003ca8034fe32b9bab8e1c03d74c972abd80dccf84a302...
4,0671515001 0675139006 0684080001 0639192002 06...,003f1bdc9212bd81fd5ec35bf7285580103cd8a469340e...
...,...,...
9995,0700819006 0568858001 0524825006 0549607002 06...,ffd8e32e538255ececbef268d6f04ffcdf66fe00a1b459...
9996,0620081001 0690478001 0565379021 0634320006 06...,ffed71450acc902268fc4b99dfdc28f1a22583f573863b...
9997,0648719001 0648719001 0573937001 0573937001 05...,fffabaebcc10efa0e613b58de37901e04fa25a2f90a0a8...
9998,0573937001 0610776023 0294008002 0448515001 04...,fffae8eb3a282d8c43c77dd2ca0621703b71e90904dfde...
